### Getting %P&L


In [6]:
import requests
import json
from time import sleep
# Define the symbol and URL
call = "P-BTC-59000-140824"
# put = "P-BTC-66600-240724"
buy = 28.70
url = f"https://cdn.india.deltaex.org/v2/tickers/{call}"
response = requests.get(url)
ticker_info = response.json()
call_current = float(ticker_info['result']['mark_price'])
print(call_current)
# Print the JSON response with indentation for better readability
json_dumps = json.dumps(ticker_info['result']['mark_price'], indent=4)
print(json_dumps)

513.48690235
"513.48690235"


### Getting iv 

In [1]:
import requests
import json
from time import sleep
# Define the symbol and URL
call = "C-BTC-66600-240724"
put = "P-BTC-66600-240724"
def get_iv(symbol):

    url = f"https://cdn.india.deltaex.org/v2/tickers/{symbol}"
    response = requests.get(url)
    ticker_info = response.json()
    # Print the JSON response with indentation for better readability
    json_dumps = json.dumps(ticker_info['result'], indent=4)
    mark_iv = float(ticker_info['result']['quotes']['mark_iv'])
    return mark_iv


call_iv = get_iv(call)
put_iv = get_iv(put)
ATM_iv = (call_iv + put_iv)/2
print(f"Atm IV: {ATM_iv*100}")


Atm IV: 49.80612599999999


In [3]:
import requests
import math
def find_atm(symbol,gap):
        

        response = requests.get("https://cdn.india.deltaex.org/v2/tickers" + f"/{symbol}")
        ticker_info = response.json()
        btc_price = ticker_info['result']['close']
        atm_value =  int(math.floor(btc_price / gap)) * gap
        call_strike = atm + gap
        put_strike = atm - gap
        return(atm_value,call_strike,put_strike)

find_atm("BTCUSD",200)




ConnectTimeout: HTTPSConnectionPool(host='cdn.india.deltaex.org', port=443): Max retries exceeded with url: /v2/tickers/BTCUSD (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000223F4E95B80>, 'Connection to cdn.india.deltaex.org timed out. (connect timeout=None)'))

### Products 


In [11]:
import requests
import json

def get_product_id(option, symbol, strike, date):
    # Define the target symbol

# Fetch products
    response = requests.get('https://cdn.india.deltaex.org/v2/products')
    products = response.json()

    # Find product ID for "C-BTC-61000-190824"
    symbol = f"{option}-{symbol}-{strike}-{date}"
    product_id = None

    for product in products.get('result', []):
        if product.get('symbol') == symbol:
            product_id = product.get('id')
            break

    if product_id:
        print(f"The product ID for '{symbol}' is: {product_id}")
    else:
        print(f"Product '{symbol}' not found.")

    return product_id

product_id = get_product_id("P","BTC",put_strike,"090924")

The product ID for 'P-BTC-54000-090924' is: 36308


### Really placing order

In [7]:
import requests
import time
import hmac
import hashlib
import json
from time import sleep
from dotenv import load_dotenv
import os
from pymongo import MongoClient
import math

# Load environment variables from the .env file
load_dotenv()
mongo_uri = os.getenv('MONGO_URI')
api_key = os.getenv('API_KEY')
api_secret = os.getenv('API_SECRET')

# Set up MongoDB connection
client = MongoClient(mongo_uri)
db = client['trading_db']
orders_collection = db['orders']

# Function to find the ATM (At-The-Money) value, call strike, and put strike
def find_atm(symbol, gap):
    response = requests.get(f"https://cdn.india.deltaex.org/v2/tickers/{symbol}")
    ticker_info = response.json()
    
    btc_price = ticker_info['result']['close']
    if gap == 0:
    atm_value = int(math.floor(btc_price / gap)) * gap
    
    call_strike = atm_value + gap
    put_strike = atm_value - gap
    
    return atm_value, call_strike, put_strike

# Function to fetch the product ID for the given option, symbol, strike, and date
def get_product_id(option, symbol, strike, date):
    response = requests.get('https://cdn.india.deltaex.org/v2/products')
    products = response.json()

    symbol_format = f"{option}-{symbol}-{strike}-{date}"
    product_id = None

    # Find the product matching the given symbol format
    for product in products.get('result', []):
        if product.get('symbol') == symbol_format:
            product_id = product.get('id')
            break

    if product_id:
        print(f"The product ID for '{symbol_format}' is: {product_id}")
    else:
        print(f"Product '{symbol_format}' not found.")

    return product_id

# Function to generate the HMAC signature for API authentication
def generate_signature(method, endpoint, payload):
    timestamp = str(int(time.time()))
    signature_data = method + timestamp + endpoint + payload
    message = bytes(signature_data, 'utf-8')
    secret = bytes(api_secret, 'utf-8')
    hash = hmac.new(secret, message, hashlib.sha256)
    return hash.hexdigest(), timestamp

# Function to create an order on the exchange
def create_order(product_id, size, side):
    method = 'POST'
    endpoint = '/v2/orders'
    
    order_data = {
        'product_id': product_id,
        'size': size,
        'order_type': 'market_order',
        'side': side
    }

    body = json.dumps(order_data, separators=(',', ':'))
    signature, timestamp = generate_signature(method, endpoint, body)

    # Set up headers for the API request
    headers = {
        'api-key': api_key,
        'signature': signature,
        'timestamp': timestamp,
        'Content-Type': 'application/json'
    }

    max_retries = 5
    retry_delay = 1

    # Retry mechanism for placing the order
    for i in range(max_retries):
        response = requests.post(f'https://cdn.india.deltaex.org{endpoint}', headers=headers, data=body)
        order_response = response.json()

        # Insert order information into MongoDB
        order_info = {
            'timestamp': time.time(),
            'request_data': order_data,
            'response_data': order_response
        }
        orders_collection.insert_one(order_info)

        print(order_response)

        # Exit loop if order is placed successfully
        if order_response.get('success'):
            print("Order placed successfully!")
            break

        # Exponential backoff for retries
        sleep(retry_delay)
        retry_delay *= 2

    if not order_response.get('success'):
        print("Failed to place the order after multiple attempts.")

# Example usage of the functions

# Find ATM, call, and put strikes for BTCUSD with a gap of 200
atm_value, call_strike, put_strike = find_atm("ETHUSD", 1)

# Get product IDs for call and put options
call_product_id = get_product_id("C", "ETH", call_strike, "100924")
put_product_id = get_product_id("P", "ETH", put_strike, "100924")

# Place buy orders for the call and put options
if call_product_id:
    create_order(call_product_id, 1, 'buy')

if put_product_id:
    create_order(put_product_id, 1, 'buy')


Product 'C-ETH-2311-100924' not found.
Product 'P-ETH-2309-100924' not found.


### SELLING

In [ ]:
import requests
import time
import hmac
import hashlib
import json
from time import sleep
from dotenv import load_dotenv
import os
from pymongo import MongoClient
import math

# Load environment variables from the .env file
load_dotenv()
mongo_uri = os.getenv('MONGO_URI')
api_key = os.getenv('API_KEY')
api_secret = os.getenv('API_SECRET')

# Set up MongoDB connection
client = MongoClient(mongo_uri)
db = client['trading_db']
orders_collection = db['orders']

# Function to find the ATM (At-The-Money) value, call strike, and put strike
def find_atm(symbol, gap):
    response = requests.get(f"https://cdn.india.deltaex.org/v2/tickers/{symbol}")
    ticker_info = response.json()
    
    btc_price = ticker_info['result']['close']
    if gap == 0:
    atm_value = int(math.floor(btc_price / gap)) * gap
    
    call_strike = atm_value + gap
    put_strike = atm_value - gap
    
    return atm_value, call_strike, put_strike

# Function to fetch the product ID for the given option, symbol, strike, and date
def get_product_id(option, symbol, strike, date):
    response = requests.get('https://cdn.india.deltaex.org/v2/products')
    products = response.json()

    symbol_format = f"{option}-{symbol}-{strike}-{date}"
    product_id = None

    # Find the product matching the given symbol format
    for product in products.get('result', []):
        if product.get('symbol') == symbol_format:
            product_id = product.get('id')
            break

    if product_id:
        print(f"The product ID for '{symbol_format}' is: {product_id}")
    else:
        print(f"Product '{symbol_format}' not found.")

    return product_id

# Function to generate the HMAC signature for API authentication
def generate_signature(method, endpoint, payload):
    timestamp = str(int(time.time()))
    signature_data = method + timestamp + endpoint + payload
    message = bytes(signature_data, 'utf-8')
    secret = bytes(api_secret, 'utf-8')
    hash = hmac.new(secret, message, hashlib.sha256)
    return hash.hexdigest(), timestamp

# Function to create an order on the exchange
def create_order(product_id, size, side):
    method = 'POST'
    endpoint = '/v2/orders'
    
    order_data = {
        'product_id': product_id,
        'size': size,
        'order_type': 'market_order',
        'side': side
    }

    body = json.dumps(order_data, separators=(',', ':'))
    signature, timestamp = generate_signature(method, endpoint, body)

    # Set up headers for the API request
    headers = {
        'api-key': api_key,
        'signature': signature,
        'timestamp': timestamp,
        'Content-Type': 'application/json'
    }

    max_retries = 5
    retry_delay = 1

    # Retry mechanism for placing the order
    for i in range(max_retries):
        response = requests.post(f'https://cdn.india.deltaex.org{endpoint}', headers=headers, data=body)
        order_response = response.json()

        # Insert order information into MongoDB
        order_info = {
            'timestamp': time.time(),
            'request_data': order_data,
            'response_data': order_response
        }
        orders_collection.insert_one(order_info)

        print(order_response)

        # Exit loop if order is placed successfully
        if order_response.get('success'):
            print("Order placed successfully!")
            break

        # Exponential backoff for retries
        sleep(retry_delay)
        retry_delay *= 2

    if not order_response.get('success'):
        print("Failed to place the order after multiple attempts.")

# Example usage of the functions

# Find ATM, call, and put strikes for BTCUSD with a gap of 200
atm_value, call_strike, put_strike = find_atm("ETHUSD", 1)


# Get product IDs for call and put options
# eth_call_product_id = get_product_id("C", "ETH", '2320', "100924")
# eth_put_product_id = get_product_id("P", "ETH", '2280', "100924")
# btc_call_product_id = get_product_id("C", "BTC", '55400', "100924")
# btc_put_product_id = get_product_id("P", "BTC", '55000', "100924")
# Place buy orders for the call and put options
if eth_call_product_id:
    create_order(call_product_id, 5, 'sell')

if eth_put_product_id:
    create_order(put_product_id, 5, 'sell')
if btc_call_product_id:
    create_order(call_product_id, 3, 'sell')

if btc_put_product_id:
    create_order(put_product_id, 3, 'sell')

In [1]:
from datetime import datetime

order_time = "10-09-2024 00:58:00"
current_time = datetime.now()

# # Convert the string with both date and time to a datetime object
# target_time = datetime.strptime(order_time, "%d-%m-%Y %H:%M:%S")

# # Calculate the difference in seconds
# delay_seconds = (target_time - current_time).total_seconds()

# if delay_seconds > 0:
#     print(f'yes {delay_seconds} seconds remaining')
# else:
#     print('no')
current_time

datetime.datetime(2024, 9, 18, 21, 35, 4, 281332)